In [32]:
import nltk
from nltk.corpus import wordnet
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import pandas as pd
import re

In [13]:
nltk.download('averaged_perceptron_tagger')
nltk.download('vader_lexicon')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Dasha\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Dasha\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dasha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dasha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [12]:
data_file = '../data/intermediate/filtered_data.tsv'

data = pd.read_csv(data_file, sep='\t')

data.head()

,reference,translation,ref_tox,trn_tox
0,if alkar floods her with her mental waste it w...,if alkar is flooding her with psychic waste th...,0.981983,0.014195
1,i have orders to kill her,ive got orders to put her down,0.999348,0.009402
2,im not gonna have a child with the same geneti...,im not going to breed kids with a genetic diso...,0.950956,0.035846
3,theyre all laughing at us so well kick your ass,theyre laughing at us well show you,0.999492,0.000131
4,youd probably want me to buy you some chocolat...,i suppose you want me to buy you flowers and c...,0.980341,0.000078


In [19]:
# Initialize sentiment analyzer
sia = SentimentIntensityAnalyzer()

# Function to identify negative words based on sentiment analysis
def identify_negative_words(text):
    tokenized_text = word_tokenize(text)
    pos_tags = nltk.pos_tag(tokenized_text)
    
    negative_words = []
    for word, tag in pos_tags:
        if sia.polarity_scores(word)['compound'] < 0:
            # Check if word is an adverb (often intensify negativity)
            if tag.startswith('RB') or tag.startswith('RBR') or tag.startswith('RBS'):
                negative_words.append((word, tag, 'omit'))  # Mark for omission
            else:
                negative_words.append((word, tag, 'replace'))  # Mark for replacement
    return negative_words

# Function to find synonyms
def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            if lemma.name() != word:
                synonyms.add(lemma.name().replace('_', ' '))
    return list(synonyms)

# Function to replace or omit negative words
def modify_negative_words(text, negative_words):
    tokenized_text = word_tokenize(text)
    modified_text = []
    
    for word in tokenized_text:
        negation = next((neg for neg in negative_words if neg[0] == word), None)
        if negation:
            word, tag, action = negation
            if action == 'replace':
                synonyms = get_synonyms(word)
                if synonyms:
                    word = synonyms[0]  # Just pick the first synonym for simplicity
            elif action == 'omit':
                word = ''  # Omit the word from the sentence
        
        # Add the word if it's not marked for omission
        if word:
            modified_text.append(word)
            
    return ' '.join(modified_text)

In [20]:
data = pd.DataFrame({
    'reference': [
        'I hate this shit so much.',
        'The food was fucking terrible.',
        'He is a horrible nigger.',
        'They are laughing because we will fuck up.'
    ]
})

for index, row in data.iterrows():
    sentence = row['reference']
    negative_words = identify_negative_words(sentence)
    new_sentence = modify_negative_words(sentence, negative_words)
    data.at[index, 'translation'] = new_sentence

data.head()

,reference,translation
0,I hate this shit so much.,I hatred this Irish bull so much .
1,The food was fucking terrible.,The food was fucking fearful .
2,He is a horrible nigger.,He is a horrifying spade .
3,They are laughing because we will fuck up.,They are laughing because we will make out up .


I've added feature of ommiting negative sentiments words if there are no synonyms for them. But it didn't work.

In [25]:
sia = SentimentIntensityAnalyzer()

sia = SentimentIntensityAnalyzer()

def identify_negative_words(text):
    tokenized_text = word_tokenize(text)
    pos_tags = nltk.pos_tag(tokenized_text)
    
    negative_words = []
    for word, tag in pos_tags:
        if sia.polarity_scores(word)['compound'] < 0:
            negative_words.append((word, tag))
    return negative_words

def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            if lemma.name() != word:
                synonyms.add(lemma.name().replace('_', ' '))
    return list(synonyms)

def modify_negative_words(text, negative_words):
    for word, tag in negative_words:
        synonyms = get_synonyms(word)
        non_negative_synonyms = [syn for syn in synonyms if sia.polarity_scores(syn)['compound'] >= 0]
        if non_negative_synonyms:
            text = text.replace(word, non_negative_synonyms[0], 1)
    return text

def reframe_sentence(text, negative_words):
    tokenized_text = word_tokenize(text)
    for word, tag in negative_words:
        # Check for negation patterns in the sentence
        if word.lower() in ["not", "no", "never", "n't"] and len(tokenized_text) > tokenized_text.index(word) + 1:
            # Attempt to negate the following word directly if it's an adjective or verb
            next_word = tokenized_text[tokenized_text.index(word) + 1]
            if tag.startswith('JJ') or tag.startswith('VB'):
                synonyms = get_synonyms(next_word)
                non_negative_synonyms = [syn for syn in synonyms if sia.polarity_scores(syn)['compound'] >= 0]
                if non_negative_synonyms:
                    # Replace the negation + word with a non-negative synonym
                    text = text.replace(word + ' ' + next_word, non_negative_synonyms[0], 1)
        else:
            synonyms = get_synonyms(word)
            non_negative_synonyms = [syn for syn in synonyms if sia.polarity_scores(syn)['compound'] >= 0]
            if non_negative_synonyms:
                text = text.replace(word, non_negative_synonyms[0], 1)
    return text

In [28]:
data = pd.DataFrame({
    'reference': [
        'I hate this shit so much.',
        'The food was fucking terrible.',
        'He is a horrible nigger.',
        'They are laughing because we will fuck up.'
    ]
})

for index, row in data.iterrows():
    sentence = row['reference']
    negative_words = identify_negative_words(sentence)
    
    # Step 1: Modify negative words
    modified_sentence = modify_negative_words(sentence, negative_words)
    
    # Step 2: Check the sentiment of the modified sentence
    reframed_sentence = reframe_sentence(modified_sentence, negative_words)
    data.at[index, 'translation'] = reframed_sentence

data.head()

,reference,translation
0,I hate this shit so much.,I detest this Irish bull so much.
1,The food was fucking terrible.,The food was fucking tremendous.
2,He is a horrible nigger.,He is a atrocious spade.
3,They are laughing because we will fuck up.,They are laughing because we will make out up.


I've added sentence reconstruction after pulling in synonyms. It didn't work well.

Let's try different approach.

I downloaded list of banned words from https://www.freewebheaders.com/full-list-of-bad-words-banned-by-google/ by Google. Let's try to work with this.

In [31]:
banned_words_file = '../data/external/bad-words.txt'

with open(banned_words_file, 'r') as file:
    banned_words_list = file.read().splitlines()

# Convert the list into a DataFrame
banned_df = pd.DataFrame(banned_words_list, columns=['banned_word'])

banned_df.head()

,banned_word
0,2 girls 1 cup
1,2g1c
2,4r5e
3,5h1t
4,5hit


In [42]:
def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            if lemma.name().lower() != word:
                synonyms.add(lemma.name().lower().replace('_', ' '))
    return list(synonyms)

# Define the function to identify and replace negative phrases
def detoxify_sentence(sentence, banned_words_df):
    # Lowercase the sentence to ensure proper matching
    lowered_sentence = sentence.lower()
    for banned_phrase in banned_words_df['banned_word']:
        # Check if the banned phrase is in the sentence
        if re.search(r'\b' + re.escape(banned_phrase) + r'\b', lowered_sentence, flags=re.IGNORECASE):
            # Split the banned phrase to check for synonyms for single words only
            banned_words = banned_phrase.split()
            # Only find synonyms if the banned phrase is a single word
            if len(banned_words) == 1:
                synonyms = get_synonyms(banned_words[0])
                if synonyms:
                    replacement = synonyms[0]
                else:
                    replacement = ''
            else:
                # For multi-word phrases, it's more complex to find synonyms
                # So we'll opt to just remove the phrase
                replacement = ''
            # Replace the banned phrase with the replacement or remove it
            sentence = re.sub(r'\b' + re.escape(banned_phrase) + r'\b', replacement, sentence, flags=re.IGNORECASE)
    return sentence.strip()

In [47]:
data = pd.DataFrame({
    'reference': [
        'That was a stupid idea, you ass dlck!',
        'Motherfucka! a55! style doggy!',
        'He is such a jerk when he drinks.',
        'She is a slutty fox in her debates.',
        'They are laughing because we will fuck up.',
    ]
})

# Apply the detoxify_sentence function to the 'reference' column
data['translation'] = data['reference'].apply(lambda x: detoxify_sentence(x, banned_df))

data.head()

,reference,translation
0,"That was a stupid idea, you ass dlck!","That was a pudden-head idea, you nates !"
1,Motherfucka! a55! style doggy!,! ! !
2,He is such a jerk when he drinks.,He is such a twitch when he drinks.
3,She is a slutty fox in her debates.,She is a slutty fox in her debates.
4,They are laughing because we will fuck up.,They are laughing because we will make out up.
